In [0]:
# FUNÇÕES UTILIZADAS ==> (date_format) / (orderBy) / (substring) / (rpad) / (overlay) / (regexp_replace)

In [0]:
# IMPORTANDO AS BIBLIOTECAS DO SPARK

import pyspark.sql.functions 
from pyspark.sql.types import StructType , StructField , StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
# CRIANDO SESSÃO DO SPARK 

spark = (SparkSession.builder
        .master("local[*]")
        .appName("Pyspark-2")
        .getOrCreate())

In [0]:
# CRIANDO DATA FRAME ==> DADOS

data = (['1' , 'José', 'Anápolis' , 'São Paulo' , '01-09-1900'],
       ['2' , 'Igor', 'Anápolis' , 'São Paulo' , '11-09-1977'],
       ['3' , 'Leonardo', 'Anápolis' , 'São Paulo' , '21-12-2000'],
       ['4' , 'Humberto', 'Pato Branco' , 'Rio Grande do Sul' , '13-11-1964'],
       ['5' , 'Isaías', 'Pato Branco' , 'Rio Grande do Sul' , '07-07-2002'],
       ['6' , 'Lucas', 'Tauá' , 'Ceará' , '05-09-1984'],
       ['7' , 'Cláudio', 'Belo Horizonte' , 'Minas Gerais' , '07-09-1977'],
       ['8' , 'Hélio', 'Contagem' , 'Belo Horizonte' , '24-07-1976'],
       ['9' , 'Guilherme', 'Recife' , 'Pernambuco' , '06-11-1996']
       )

In [0]:
# COLUNAS DO DATAFRAME 
colunas = (StructType([
    
    StructField('cod_cliente', StringType(), True),
    StructField('nome', StringType(), True),
    StructField('municipio', StringType(), True),
    StructField('estado', StringType(), True),
    StructField('data_nasc', StringType(), True)
]))

In [0]:
## CRIANDO DATA FRAME FINAL 

df = spark.createDataFrame(data = data , schema = colunas)

df.display()

cod_cliente,nome,municipio,estado,data_nasc
1,José,Anápolis,São Paulo,01-09-1900
2,Igor,Anápolis,São Paulo,11-09-1977
3,Leonardo,Anápolis,São Paulo,21-12-2000
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002
6,Lucas,Tauá,Ceará,05-09-1984
7,Cláudio,Belo Horizonte,Minas Gerais,07-09-1977
8,Hélio,Contagem,Belo Horizonte,24-07-1976
9,Guilherme,Recife,Pernambuco,06-11-1996


In [0]:
# 1) CRIAR UMA COLUNA PARA (ANOS) E ORDERNAR DO MAIS VELHO PARA O MAIS NOVO 

df = df.withColumn('anos' , date_format(current_timestamp(), 'yyyy').cast('integer') - substring((df.data_nasc), 7,4).cast('integer'))

# TRANSFORMANDO DO MAIOR PARA O MENOR 
#df = df.orderBy(col('anos').desc()).show()
df.select('anos').orderBy(col('anos').desc()).show()  


+----+
|anos|
+----+
| 122|
|  58|
|  46|
|  45|
|  45|
|  38|
|  26|
|  22|
|  20|
+----+



In [0]:
# 2) NA COLUNA (COD_CLIENTE) COLOCAR UM ("- " A DIREITA DEPOIS DO NUMERO)

df = df.withColumn('cod_cliente', rpad('cod_cliente', 2, '- '))


df.show()

+-----------+---------+--------------+-----------------+----------+----+
|cod_cliente|     nome|     municipio|           estado| data_nasc|anos|
+-----------+---------+--------------+-----------------+----------+----+
|         1-|     José|      Anápolis|        São Paulo|01-09-1900| 122|
|         2-|     Igor|      Anápolis|        São Paulo|11-09-1977|  45|
|         3-| Leonardo|      Anápolis|        São Paulo|21-12-2000|  22|
|         4-| Humberto|   Pato Branco|Rio Grande do Sul|13-11-1964|  58|
|         5-|   Isaías|   Pato Branco|Rio Grande do Sul|07-07-2002|  20|
|         6-|    Lucas|          Tauá|            Ceará|05-09-1984|  38|
|         7-|  Cláudio|Belo Horizonte|     Minas Gerais|07-09-1977|  45|
|         8-|    Hélio|      Contagem|   Belo Horizonte|24-07-1976|  46|
|         9-|Guilherme|        Recife|       Pernambuco|06-11-1996|  26|
+-----------+---------+--------------+-----------------+----------+----+



In [0]:
# 3) CRIAR UMA COLUNA UNINDO A COLUNA (COD_CLIENTE + NOME)

df = df.withColumn('cod+nome_client' , overlay('cod_cliente','nome',10))

df.show()

+-----------+---------+--------------+-----------------+----------+----+---------------+
|cod_cliente|     nome|     municipio|           estado| data_nasc|anos|cod+nome_client|
+-----------+---------+--------------+-----------------+----------+----+---------------+
|         1-|     José|      Anápolis|        São Paulo|01-09-1900| 122|         1-José|
|         2-|     Igor|      Anápolis|        São Paulo|11-09-1977|  45|         2-Igor|
|         3-| Leonardo|      Anápolis|        São Paulo|21-12-2000|  22|     3-Leonardo|
|         4-| Humberto|   Pato Branco|Rio Grande do Sul|13-11-1964|  58|     4-Humberto|
|         5-|   Isaías|   Pato Branco|Rio Grande do Sul|07-07-2002|  20|       5-Isaías|
|         6-|    Lucas|          Tauá|            Ceará|05-09-1984|  38|        6-Lucas|
|         7-|  Cláudio|Belo Horizonte|     Minas Gerais|07-09-1977|  45|      7-Cláudio|
|         8-|    Hélio|      Contagem|   Belo Horizonte|24-07-1976|  46|        8-Hélio|
|         9-|Guilherm

In [0]:
# 4) ADICIONAR UMA COLUNA COM A DATA ATUAL DO SISTEMA ==> dd-mm-yyyy hh:mm:ss

df = df.withColumn('data_atual', date_format(current_timestamp(), 'yyyy-MM-dd HH:mm:ss'))

df.show()


+-----------+---------+--------------+-----------------+----------+----+---------------+-------------------+
|cod_cliente|     nome|     municipio|           estado| data_nasc|anos|cod+nome_client|         data_atual|
+-----------+---------+--------------+-----------------+----------+----+---------------+-------------------+
|         1-|     José|      Anápolis|        São Paulo|01-09-1900| 122|         1-José|2022-08-13 20:25:47|
|         2-|     Igor|      Anápolis|        São Paulo|11-09-1977|  45|         2-Igor|2022-08-13 20:25:47|
|         3-| Leonardo|      Anápolis|        São Paulo|21-12-2000|  22|     3-Leonardo|2022-08-13 20:25:47|
|         4-| Humberto|   Pato Branco|Rio Grande do Sul|13-11-1964|  58|     4-Humberto|2022-08-13 20:25:47|
|         5-|   Isaías|   Pato Branco|Rio Grande do Sul|07-07-2002|  20|       5-Isaías|2022-08-13 20:25:47|
|         6-|    Lucas|          Tauá|            Ceará|05-09-1984|  38|        6-Lucas|2022-08-13 20:25:47|
|         7-|  Cláu

In [0]:
# 5) REMOVER OS ACENTOS DAS COLUNAS ==> NOME, MUNICIPIO , ESTADO 

df = df.withColumn('nome' , regexp_replace('nome' , 'é' , 'e'))\
                   .withColumn('nome' , regexp_replace('nome' , 'á' , 'a'))\
                   .withColumn('nome' , regexp_replace('nome' , 'í' , 'i'))\
                   .withColumn('municipio' , regexp_replace('municipio' , 'á' , 'a'))\
                   .withColumn('estado' , regexp_replace('estado' , 'ã' , 'a'))\
                   .withColumn('estado' , regexp_replace('estado' , 'á' , 'a'))

df.show()

+-----------+---------+--------------+-----------------+----------+----+---------------+-------------------+
|cod_cliente|     nome|     municipio|           estado| data_nasc|anos|cod+nome_client|         data_atual|
+-----------+---------+--------------+-----------------+----------+----+---------------+-------------------+
|         1-|     Jose|      Anapolis|        Sao Paulo|01-09-1900| 122|         1-José|2022-08-13 20:25:48|
|         2-|     Igor|      Anapolis|        Sao Paulo|11-09-1977|  45|         2-Igor|2022-08-13 20:25:48|
|         3-| Leonardo|      Anapolis|        Sao Paulo|21-12-2000|  22|     3-Leonardo|2022-08-13 20:25:48|
|         4-| Humberto|   Pato Branco|Rio Grande do Sul|13-11-1964|  58|     4-Humberto|2022-08-13 20:25:48|
|         5-|   Isaias|   Pato Branco|Rio Grande do Sul|07-07-2002|  20|       5-Isaías|2022-08-13 20:25:48|
|         6-|    Lucas|          Taua|            Ceara|05-09-1984|  38|        6-Lucas|2022-08-13 20:25:48|
|         7-|  Clau